In [1]:
import numpy as np
import pandas as pd
import subprocess
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [65]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
submission = pd.read_csv('../input/sample_submission.csv.zip')

In [8]:
train.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [9]:
test.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [11]:
np.random.seed(123)
index = list(train.index)
print(index[0:10])
np.random.shuffle(index)
print(index[0:10])
train = train.iloc[index]
'train = train.iloc[np.random.permutation(len(train))]'

## set test loss to NaN
test['loss'] = np.nan

## response and IDs
y = np.log(train['loss'].values+200)
id_train = train['id'].values
id_test = test['id'].values

## stack train test
ntrain = train.shape[0]
tr_te = pd.concat((train, test), axis = 0)

## Preprocessing and transforming to sparse data
sparse_data = []

f_cat = [f for f in tr_te.columns if 'cat' in f]
for f in f_cat:
    dummy = pd.get_dummies(tr_te[f].astype('category'))
    tmp = csr_matrix(dummy)
    sparse_data.append(tmp)

f_num = [f for f in tr_te.columns if 'cont' in f]
scaler = StandardScaler()
tmp = csr_matrix(scaler.fit_transform(tr_te[f_num]))
sparse_data.append(tmp)

del(tr_te, train, test)

## sparse train and test data
xtr_te = hstack(sparse_data, format = 'csr')
xtrain = xtr_te[:ntrain, :]
xtest = xtr_te[ntrain:, :]

print('Dim train', xtrain.shape)
print('Dim test', xtest.shape)

del(xtr_te, sparse_data, tmp)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[33898, 43407, 3458, 100139, 60086, 117290, 89806, 7751, 164059, 143297]
Dim train (188318, 1190)
Dim test (125546, 1190)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, PReLU

In [17]:
y

array([7.56987608, 7.84829365, 7.41602364, ..., 7.33236921, 9.17228019,
       6.37051723])

In [18]:
xtrain

<188318x1190 sparse matrix of type '<class 'numpy.float64'>'
	with 24481340 stored elements in Compressed Sparse Row format>

In [66]:
# Separate features and targets
X = xtrain
Y = y

# Set the input shape
input_shape = (xtrain.shape[1],)
print(f'Feature shape: {input_shape}')

# Create the model
model = Sequential()
model.add(Dense(16, input_shape=input_shape, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))


Feature shape: (1190,)


In [67]:
X.toarray()

array([[ 0.        ,  1.        ,  1.        , ..., -0.95914517,
        -0.61232072, -1.26329393],
       [ 1.        ,  0.        ,  1.        , ...,  0.39395194,
        -0.71240011, -1.1963163 ],
       [ 1.        ,  0.        ,  1.        , ...,  0.91652738,
         0.94750019, -0.90385702],
       ...,
       [ 0.        ,  1.        ,  1.        , ..., -0.83567771,
        -0.01839175,  1.55127219],
       [ 1.        ,  0.        ,  0.        , ..., -0.23903515,
        -0.80958157, -0.56602101],
       [ 0.        ,  1.        ,  1.        , ...,  0.39395194,
         1.53407398, -0.95125139]])

In [68]:
# Configure the model and start training
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.fit(X.toarray(), Y, epochs=20, batch_size=1024, verbose=1, validation_split=0.2)

Epoch 1/20
148/148 [==============================] - 1s 4ms/step - loss: 1.6447 - mae: 1.6447 - val_loss: 0.4246 - val_mae: 0.4246
Epoch 2/20
148/148 [==============================] - 0s 3ms/step - loss: 0.4045 - mae: 0.4045 - val_loss: 0.3957 - val_mae: 0.3957
Epoch 3/20
148/148 [==============================] - 0s 3ms/step - loss: 0.3892 - mae: 0.3892 - val_loss: 0.3896 - val_mae: 0.3896
Epoch 4/20
148/148 [==============================] - 0s 3ms/step - loss: 0.3846 - mae: 0.3846 - val_loss: 0.3862 - val_mae: 0.3862
Epoch 5/20
148/148 [==============================] - 0s 3ms/step - loss: 0.3826 - mae: 0.3826 - val_loss: 0.3844 - val_mae: 0.3844
Epoch 6/20
148/148 [==============================] - 0s 3ms/step - loss: 0.3815 - mae: 0.3815 - val_loss: 0.3839 - val_mae: 0.3839
Epoch 7/20
148/148 [==============================] - 0s 3ms/step - loss: 0.3809 - mae: 0.3809 - val_loss: 0.3842 - val_mae: 0.3842
Epoch 8/20
148/148 [==============================] - 0s 3ms/step - loss: 0.

In [70]:
preds = np.exp(model.predict(xtest))-200
preds

array([[ 1296.1677],
       [ 1648.4943],
       [11054.149 ],
       ...,
       [ 2436.859 ],
       [  904.2169],
       [ 3309.8083]], dtype=float32)

In [72]:
submission['loss'] = preds
submission.to_csv('../submissions/sub_simple_mlp_0.csv', index=False)
submission.head()

,id,loss
0,4,1296.167725
1,6,1648.494263
2,9,11054.149414
3,12,5829.130859
4,15,771.655334


In [74]:
## neural net
def nn_model():
    model = Sequential()
    
    model.add(Dense(400, input_dim = xtrain.shape[1], kernel_initializer = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.4))
        
    model.add(Dense(200, kernel_initializer = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(0.2))
    
    model.add(Dense(50, kernel_initializer = 'he_normal'))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(0.2))
    
    model.add(Dense(1, kernel_initializer = 'he_normal'))
    model.compile(loss = 'mae', optimizer = 'adadelta')
    return(model)

In [78]:
model = nn_model()

In [79]:
model.fit(X.toarray(), Y, epochs=600, batch_size=1024, verbose=1, validation_split=0.2)

Epoch 1/600
148/148 [==============================] - 1s 6ms/step - loss: 7.8005 - val_loss: 8.2878
Epoch 2/600
148/148 [==============================] - 1s 4ms/step - loss: 7.7985 - val_loss: 8.3257
Epoch 3/600
148/148 [==============================] - 1s 5ms/step - loss: 7.7923 - val_loss: 8.1962
Epoch 4/600
148/148 [==============================] - 1s 5ms/step - loss: 7.7952 - val_loss: 8.0987
Epoch 5/600
148/148 [==============================] - 1s 6ms/step - loss: 7.7886 - val_loss: 8.0501
Epoch 6/600
148/148 [==============================] - 1s 6ms/step - loss: 7.7809 - val_loss: 8.0358
Epoch 7/600
148/148 [==============================] - 1s 5ms/step - loss: 7.7810 - val_loss: 8.0325
Epoch 8/600
148/148 [==============================] - 1s 5ms/step - loss: 7.7800 - val_loss: 8.0283
Epoch 9/600
148/148 [==============================] - 1s 5ms/step - loss: 7.7713 - val_loss: 8.0263
Epoch 10/600
148/148 [==============================] - 1s 5ms/step - loss: 7.7709 - val_lo

In [80]:
preds = np.exp(model.predict(xtest))-200
preds

array([[ 840.42236],
       [1138.6089 ],
       [2698.374  ],
       ...,
       [3150.0344 ],
       [ 999.2999 ],
       [2696.4177 ]], dtype=float32)

In [81]:
submission['loss'] = preds
submission.to_csv('../submissions/sub_cmplx_mlp_0.csv', index=False)
submission.head()

,id,loss
0,4,840.422363
1,6,1138.608887
2,9,2698.374023
3,12,4880.146484
4,15,738.596924


In [82]:
# Configure the model and start training
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.fit(X.toarray(), Y, epochs=20, batch_size=100, verbose=1, validation_split=0.2)

Epoch 1/20
1507/1507 [==============================] - 8s 5ms/step - loss: 0.7430 - mae: 0.7430 - val_loss: 0.4228 - val_mae: 0.4228
Epoch 2/20
1507/1507 [==============================] - 8s 6ms/step - loss: 0.5887 - mae: 0.5887 - val_loss: 0.3990 - val_mae: 0.3990
Epoch 3/20
1507/1507 [==============================] - 8s 5ms/step - loss: 0.5452 - mae: 0.5452 - val_loss: 0.3848 - val_mae: 0.3848
Epoch 4/20
1507/1507 [==============================] - 8s 5ms/step - loss: 0.5183 - mae: 0.5183 - val_loss: 0.3968 - val_mae: 0.3968
Epoch 5/20
1507/1507 [==============================] - 8s 5ms/step - loss: 0.4977 - mae: 0.4977 - val_loss: 0.3912 - val_mae: 0.3912
Epoch 6/20
1507/1507 [==============================] - 8s 5ms/step - loss: 0.4814 - mae: 0.4814 - val_loss: 0.3830 - val_mae: 0.3830
Epoch 7/20
1507/1507 [==============================] - 8s 5ms/step - loss: 0.4684 - mae: 0.4684 - val_loss: 0.3834 - val_mae: 0.3834
Epoch 8/20
1507/1507 [==============================] - 8s 5ms

In [83]:
preds = np.exp(model.predict(xtest))-200
preds

array([[1631.2096],
       [1847.1476],
       [9865.234 ],
       ...,
       [2451.0864],
       [1141.5431],
       [3928.2632]], dtype=float32)

In [85]:
submission['loss'] = preds
submission.to_csv('../submissions/sub_cmplx_mlp_1.csv', index=False)
submission.head()

,id,loss
0,4,1631.209595
1,6,1847.147583
2,9,9865.234375
3,12,7891.284668
4,15,862.370850


In [87]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 400)               476400    
_________________________________________________________________
p_re_lu_15 (PReLU)           (None, 400)               400       
_________________________________________________________________
batch_normalization_15 (Batc (None, 400)               1600      
_________________________________________________________________
dropout_15 (Dropout)         (None, 400)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 200)               80200     
_________________________________________________________________
p_re_lu_16 (PReLU)           (None, 200)               200       
_________________________________________________________________
batch_normalization_16 (Batc (None, 200)             

In [88]:
# Configure the model and start training
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.fit(X.toarray(), Y, epochs=20, batch_size=50, verbose=1, validation_split=0.2)

Epoch 1/20
3014/3014 [==============================] - 15s 5ms/step - loss: 0.3920 - mae: 0.3920 - val_loss: 0.3750 - val_mae: 0.3750
Epoch 2/20
3014/3014 [==============================] - 17s 6ms/step - loss: 0.3896 - mae: 0.3896 - val_loss: 0.3837 - val_mae: 0.3837
Epoch 3/20
3014/3014 [==============================] - 13s 4ms/step - loss: 0.3871 - mae: 0.3871 - val_loss: 0.3745 - val_mae: 0.3745
Epoch 4/20
3014/3014 [==============================] - 16s 5ms/step - loss: 0.3858 - mae: 0.3858 - val_loss: 0.3749 - val_mae: 0.3749
Epoch 5/20
3014/3014 [==============================] - 15s 5ms/step - loss: 0.3842 - mae: 0.3842 - val_loss: 0.3739 - val_mae: 0.3739
Epoch 6/20
3014/3014 [==============================] - 17s 6ms/step - loss: 0.3829 - mae: 0.3829 - val_loss: 0.3783 - val_mae: 0.3783
Epoch 7/20
3014/3014 [==============================] - 13s 4ms/step - loss: 0.3815 - mae: 0.3815 - val_loss: 0.3781 - val_mae: 0.3781
Epoch 8/20
3014/3014 [==============================] -

In [89]:
preds = np.exp(model.predict(xtest))-200
submission['loss'] = preds
submission.to_csv('../submissions/sub_cmplx_mlp_2.csv', index=False)
submission.head()

,id,loss
0,4,1593.620483
1,6,1755.819824
2,9,9094.369141
3,12,8058.703125
4,15,849.200195


In [91]:
# Configure the model and start training
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.fit(X.toarray(), Y, epochs=10, batch_size=1024, verbose=1, validation_split=0.2)

Epoch 1/10
148/148 [==============================] - 4s 24ms/step - loss: 0.3697 - mae: 0.3697 - val_loss: 0.3738 - val_mae: 0.3738
Epoch 2/10
148/148 [==============================] - 6s 39ms/step - loss: 0.3668 - mae: 0.3668 - val_loss: 0.3734 - val_mae: 0.3734
Epoch 3/10
148/148 [==============================] - 4s 28ms/step - loss: 0.3651 - mae: 0.3651 - val_loss: 0.3725 - val_mae: 0.3725
Epoch 4/10
148/148 [==============================] - 6s 41ms/step - loss: 0.3649 - mae: 0.3649 - val_loss: 0.3727 - val_mae: 0.3727
Epoch 5/10
148/148 [==============================] - 5s 31ms/step - loss: 0.3636 - mae: 0.3636 - val_loss: 0.3722 - val_mae: 0.3722
Epoch 6/10
148/148 [==============================] - 5s 35ms/step - loss: 0.3629 - mae: 0.3629 - val_loss: 0.3722 - val_mae: 0.3722
Epoch 7/10
148/148 [==============================] - 5s 36ms/step - loss: 0.3626 - mae: 0.3626 - val_loss: 0.3727 - val_mae: 0.3727
Epoch 8/10
148/148 [==============================] - 4s 24ms/step - 

In [92]:
preds = np.exp(model.predict(xtest))-200
submission['loss'] = preds
submission.to_csv('../submissions/sub_cmplx_mlp_3.csv', index=False)
submission.head()

,id,loss
0,4,1480.357910
1,6,1785.122314
2,9,9858.220703
3,12,7311.822266
4,15,842.061646


In [94]:
# Configure the model and start training
model = Sequential()
model.add(Dense(16, input_shape=input_shape, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.fit(X.toarray(), Y, epochs=10, batch_size=2048, verbose=1, validation_split=0.2)

Epoch 1/10
74/74 [==============================] - 1s 7ms/step - loss: 1.9232 - mae: 1.9232 - val_loss: 0.4756 - val_mae: 0.4756
Epoch 2/10
74/74 [==============================] - 0s 4ms/step - loss: 0.4290 - mae: 0.4290 - val_loss: 0.4079 - val_mae: 0.4079
Epoch 3/10
74/74 [==============================] - 0s 4ms/step - loss: 0.4007 - mae: 0.4007 - val_loss: 0.3989 - val_mae: 0.3989
Epoch 4/10
74/74 [==============================] - 0s 4ms/step - loss: 0.3953 - mae: 0.3953 - val_loss: 0.3967 - val_mae: 0.3967
Epoch 5/10
74/74 [==============================] - 0s 4ms/step - loss: 0.3936 - mae: 0.3936 - val_loss: 0.3961 - val_mae: 0.3961
Epoch 6/10
74/74 [==============================] - 0s 4ms/step - loss: 0.3930 - mae: 0.3930 - val_loss: 0.3968 - val_mae: 0.3968
Epoch 7/10
74/74 [==============================] - 0s 4ms/step - loss: 0.3928 - mae: 0.3928 - val_loss: 0.3959 - val_mae: 0.3959
Epoch 8/10
74/74 [==============================] - 0s 4ms/step - loss: 0.3925 - mae: 0.39

In [96]:
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.fit(X.toarray(), Y, epochs=10, batch_size=128, verbose=1, validation_split=0.2)

Epoch 1/10
1177/1177 [==============================] - 2s 2ms/step - loss: 0.4007 - mae: 0.4007 - val_loss: 0.4196 - val_mae: 0.4196
Epoch 2/10
1177/1177 [==============================] - 2s 1ms/step - loss: 0.3997 - mae: 0.3997 - val_loss: 0.4180 - val_mae: 0.4180
Epoch 3/10
1177/1177 [==============================] - 2s 2ms/step - loss: 0.3987 - mae: 0.3987 - val_loss: 0.4043 - val_mae: 0.4043
Epoch 4/10
1177/1177 [==============================] - 2s 2ms/step - loss: 0.3980 - mae: 0.3980 - val_loss: 0.3987 - val_mae: 0.3987
Epoch 5/10
1177/1177 [==============================] - 2s 1ms/step - loss: 0.3987 - mae: 0.3987 - val_loss: 0.3995 - val_mae: 0.3995
Epoch 6/10
1177/1177 [==============================] - 2s 1ms/step - loss: 0.3977 - mae: 0.3977 - val_loss: 0.4009 - val_mae: 0.4009
Epoch 7/10
1177/1177 [==============================] - 2s 1ms/step - loss: 0.3973 - mae: 0.3973 - val_loss: 0.3939 - val_mae: 0.3939
Epoch 8/10
1177/1177 [==============================] - 2s 2ms

In [97]:
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
model.fit(X.toarray(), Y, epochs=20, batch_size=128, verbose=1, validation_split=0.2)

Epoch 1/20
1177/1177 [==============================] - 2s 2ms/step - loss: 0.3848 - mae: 0.3848 - val_loss: 0.3862 - val_mae: 0.3862
Epoch 2/20
1177/1177 [==============================] - 2s 1ms/step - loss: 0.3838 - mae: 0.3838 - val_loss: 0.3843 - val_mae: 0.3843
Epoch 3/20
1177/1177 [==============================] - 2s 2ms/step - loss: 0.3839 - mae: 0.3839 - val_loss: 0.3864 - val_mae: 0.3864
Epoch 4/20
1177/1177 [==============================] - 2s 2ms/step - loss: 0.3839 - mae: 0.3839 - val_loss: 0.3853 - val_mae: 0.3853
Epoch 5/20
1177/1177 [==============================] - 2s 1ms/step - loss: 0.3827 - mae: 0.3827 - val_loss: 0.3871 - val_mae: 0.3871
Epoch 6/20
1177/1177 [==============================] - 2s 2ms/step - loss: 0.3820 - mae: 0.3820 - val_loss: 0.3868 - val_mae: 0.3868
Epoch 7/20
1177/1177 [==============================] - 2s 2ms/step - loss: 0.3813 - mae: 0.3813 - val_loss: 0.3838 - val_mae: 0.3838
Epoch 8/20
1177/1177 [==============================] - 2s 2ms

In [100]:
model = nn_model()
model.fit(X.toarray(), Y, epochs=100, batch_size=1000, verbose=1, validation_split=0.2)

Epoch 1/100
151/151 [==============================] - 1s 6ms/step - loss: 7.7991 - val_loss: 8.6658
Epoch 2/100
151/151 [==============================] - 1s 4ms/step - loss: 7.8002 - val_loss: 8.6553
Epoch 3/100
151/151 [==============================] - 1s 4ms/step - loss: 7.7957 - val_loss: 8.4956
Epoch 4/100
151/151 [==============================] - 1s 5ms/step - loss: 7.7919 - val_loss: 8.3869
Epoch 5/100
151/151 [==============================] - 1s 5ms/step - loss: 7.7835 - val_loss: 8.3396
Epoch 6/100
151/151 [==============================] - 1s 6ms/step - loss: 7.7817 - val_loss: 8.3201
Epoch 7/100
151/151 [==============================] - 1s 6ms/step - loss: 7.7791 - val_loss: 8.3120
Epoch 8/100
151/151 [==============================] - 1s 5ms/step - loss: 7.7745 - val_loss: 8.3067
Epoch 9/100
151/151 [==============================] - 1s 5ms/step - loss: 7.7713 - val_loss: 8.2944
Epoch 10/100
151/151 [==============================] - 1s 5ms/step - loss: 7.7663 - val_lo

In [101]:
model.fit(X.toarray(), Y, epochs=100, batch_size=100, verbose=1, validation_split=0.2)

Epoch 1/100
1507/1507 [==============================] - 7s 5ms/step - loss: 6.6149 - val_loss: 7.1497
Epoch 2/100
1507/1507 [==============================] - 7s 4ms/step - loss: 6.4170 - val_loss: 6.9701
Epoch 3/100
1507/1507 [==============================] - 8s 5ms/step - loss: 6.2060 - val_loss: 6.7641
Epoch 4/100
1507/1507 [==============================] - 7s 5ms/step - loss: 5.9811 - val_loss: 6.5320
Epoch 5/100
1507/1507 [==============================] - 7s 5ms/step - loss: 5.7430 - val_loss: 6.2799
Epoch 6/100
1507/1507 [==============================] - 7s 5ms/step - loss: 5.4865 - val_loss: 6.0372
Epoch 7/100
1507/1507 [==============================] - 7s 4ms/step - loss: 5.2174 - val_loss: 5.7502
Epoch 8/100
1507/1507 [==============================] - 6s 4ms/step - loss: 4.9248 - val_loss: 5.4940
Epoch 9/100
1507/1507 [==============================] - 7s 4ms/step - loss: 4.6160 - val_loss: 5.1457
Epoch 10/100
1507/1507 [==============================] - 7s 5ms/step - l